In [5]:
import json

In [14]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

In [19]:
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score 4 or 5
            return Sentiment.POSITIVE

In [22]:
## Load Data

In [20]:
file_name = './data/sentiment/Books_small.json'
reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

In [21]:
reviews[5].sentiment

'POSITIVE'

In [23]:
## Prep Data

In [24]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=0.33, random_state=42)


In [27]:
print(training[0].sentiment)

POSITIVE


In [29]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

In [30]:
train_x[0]

"Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down."

In [31]:
test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [32]:
test_x[0]

"Every new Myke Cole book is better than the last, and this is no exception. If you haven't read the Shadow Ops series before start with Control Point, but go ahead and order Fortress Frontier and Breach Zone as well - you're going to want them."

In [33]:
## Bag of words vectorization

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [42]:
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

In [37]:
## Classification

In [68]:
#### Linear SVM
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[6])
#3, 5

array(['POSITIVE'], dtype='<U8')

In [46]:
#### Decision Tree

In [67]:
from sklearn.tree import DecisionTreeClassifier

clf_dtc = DecisionTreeClassifier()

clf_dtc.fit(train_x_vectors, train_y)
clf_dtc.predict(test_x_vectors[6])

array(['NEUTRAL'], dtype='<U8')

In [90]:
#### Naive Bayes

from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors, train_y)
#clf_gnb.fit(train_x_vectors, train_y)
#clf_gnb.predict(test_x_vectors[6])

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [91]:
#### Logistic Regression

from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(train_x_vectors, train_y)
clf_lr.predict(test_x_vectors[6])

C:\Applics\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

In [86]:
for id in range(1000):
    txt = test_x[id]
    scr = test_y[id]

    if(scr == 'NEGATIVE'):
        print(f'({id})', scr, ' / ', txt)
        print('Support Vector Machine: ', clf_svm.predict(test_x_vectors[id]))
        print('Decision Tree: ', clf_dtc.predict(test_x_vectors[id]))
        print('-------')

(35) NEGATIVE  /  I guess I am not intellectual enough to appreciate this book.I have read every Larry McMurtry book on the old west ( He won the Pulitzer for Lonesome Dove so is no amateur here ) and enjoyed every minute ... probably because I liked the characters.&#34;Onion&#34; is the single most annoying character in this book and sadly he/she is the chosen narrator ... for me, an unreliable and unlikeable one.The book also needed an editor ... it went in pointless circles and could have been much more interesting with about 100 less pages of repetition.
Support Vector Machine:  ['POSITIVE']
Decision Tree:  ['NEGATIVE']
-------
(55) NEGATIVE  /  I read a sample and it seemed ok.  The price was pretty high.  Took a chance and sad that I did.  Main characters were total idiots. Felt like my kid wrote his S***.  Refund please!
Support Vector Machine:  ['NEUTRAL']
Decision Tree:  ['POSITIVE']
-------
(63) NEGATIVE  /  I did not get past three pages of this book because the f bomb was u

IndexError: list index out of range

In [95]:
## Evaluation

### Mean accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dtc.score(test_x_vectors, test_y))
#print(clf_gbn.score(test_x_vectors, test_y))
print(clf_lr.score(test_x_vectors, test_y))

0.8242424242424242
0.7484848484848485
0.8303030303030303


In [101]:
### F1 Score

from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_dtc.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_lr.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

[0.91319444 0.21052632 0.22222222]
[0.85765125 0.15151515 0.0625    ]
[0.91370558 0.12244898 0.1       ]


In [107]:
train_y.count(Sentiment.NEGATIVE)

47